# Exercice d’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est composé de données du Titanic. Il est stocké dans un fichier csv, nommé titanic.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [4]:
titanic = pd.read_csv("./Data/titanic.csv")

## Transformation des données

Etudiez les données et tranformer-les si nécessaire

In [5]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
titanic["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [8]:
titanic.describe(include="object")

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Gustafsson, Mr. Anders Vilhelm",male,CA. 2343,G6,S
freq,1,577,7,4,644


## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [9]:
# les méthodes de prétraitement
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# on utilisera un pipeline pour enchaîner les traitements
from sklearn.pipeline import Pipeline

In [10]:
# on définit les colonnes et les transformations pour 
# les colonnes quantitatives
        
col_quanti = ['Age', 'SibSp', 'Parch', 'Fare']

transfo_quanti = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='median')),
    ('standard', StandardScaler())])

# on définit les colonnes et les transformations pour
# les variables qualitatives
col_quali = ['Pclass', 'Sex', 'Embarked']

transfo_quali = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='constant', fill_value='manquant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# on définit l'objet de la classe ColumnTransformer
# qui va permettre d'appliquer toutes les étapes
preparation = ColumnTransformer(
    transformers=[
        ('quanti', transfo_quanti , col_quanti),
        ('quali', transfo_quali , col_quali)])

In [11]:
titanic_transfo = preparation.fit_transform(titanic)

In [12]:
pd.DataFrame(titanic_transfo)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.565736,0.432793,-0.473674,-0.502445,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.663861,0.432793,-0.473674,0.786845,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,-0.258337,-0.474545,-0.473674,-0.488854,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.433312,0.432793,-0.473674,0.420730,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.433312,-0.474545,-0.473674,-0.486337,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.181487,-0.474545,-0.473674,-0.386671,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,-0.796286,-0.474545,-0.473674,-0.044381,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
888,-0.104637,0.432793,2.008933,-0.176263,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
889,-0.258337,-0.474545,-0.473674,-0.044381,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


## Prédire la survie

Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [13]:
x = titanic_transfo
y = titanic["Survived"]

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [14]:
# on importe la fonction
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

In [15]:
assert x_train.shape[0] == y_train.shape[0]
print("Bien séparé !")

Bien séparé !


Ainsi les deux échantillons _train et _test ont la même distribution

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle GBM
```python
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
```

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [17]:
modele_rf = RandomForestClassifier(n_estimators=100)
modele_knn = KNeighborsClassifier(n_neighbors=10)
modele_gbm = GradientBoostingClassifier()


Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [18]:
modele_rf.fit(x_train,y_train)
modele_knn.fit(x_train,y_train)
modele_gbm.fit(x_train,y_train)

GradientBoostingClassifier()

Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [19]:
modele_gbm.feature_importances_

array([0.12347769, 0.06528606, 0.0114729 , 0.18427627, 0.016757  ,
       0.00203078, 0.14876091, 0.12472279, 0.31012238, 0.00272465,
       0.00070538, 0.0096632 , 0.        ])

Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [20]:
y_predict_rf = modele_rf.predict(x_test)
y_predict_knn = modele_knn.predict(x_test)
y_predict_gbm = modele_gbm.predict(x_test)

On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [21]:
from sklearn.metrics import accuracy_score, recall_score


accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
accuracy_modele_gbm = accuracy_score(y_test,y_predict_gbm)
print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" %(accuracy_modele_knn))
print("Pourcentage de bien classés pour le modèle GBM :%.3f" %(accuracy_modele_gbm))

Pourcentage de bien classés pour le modèle RF : 0.762
Pourcentage de bien classés pour le modèle kNN :0.803
Pourcentage de bien classés pour le modèle GBM :0.821


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
confusion_matrix_gbm=confusion_matrix(y_test,y_predict_gbm)
print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_gbm, sep="\n")

Matrice de confusion pour le modèle RF :
[[117  21]
 [ 32  53]]
Matrice de confusion pour le modèle kNN :
[[125  13]
 [ 31  54]]
Matrice de confusion pour le modèle kNN :
[[125  13]
 [ 27  58]]


## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [23]:
from sklearn.metrics import roc_auc_score

auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])
auc_modele_gbm=roc_auc_score(y_test,modele_gbm.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
print("Aire sous la courbe ROC pour le modèle GBM :" ,auc_modele_gbm)

Aire sous la courbe ROC pour le modèle RF : 0.8391304347826087
Aire sous la courbe ROC pour le modèle kNN : 0.8408780903665815
Aire sous la courbe ROC pour le modèle GBM : 0.8652173913043478


## Aller plus loin l’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [24]:
from sklearn.model_selection import GridSearchCV

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [25]:
dico_param= {"max_depth":[3,5,7,10], "n_estimators":[10,20,50,100]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [26]:
recherche_hyper = GridSearchCV(RandomForestClassifier(), 
                               dico_param, 
                               scoring="accuracy",cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [27]:
recherche_hyper.fit(x_train, y_train)
pass

In [28]:
print(recherche_hyper.best_params_)

{'max_depth': 10, 'n_estimators': 20}


In [29]:
print(recherche_hyper.best_score_)

0.8248569184154416


**Exercice :**

Effectuez le même travail avec GBM

In [30]:
dico_param= {"max_depth":[None, 2,3,5], "n_estimators":[10,50,100]}
recherche_hyper_gbm = GridSearchCV(GradientBoostingClassifier(),
                                  dico_param,
                               scoring="accuracy",cv=5)


In [31]:
recherche_hyper_gbm.fit(x_train, y_train)
pass

In [32]:
print(recherche_hyper_gbm.best_params_)

{'max_depth': 3, 'n_estimators': 10}


In [33]:
print(recherche_hyper_gbm.best_score_)

0.8412860509482663


# Quel modèle est le meilleur ?

Décrire le modèle que vous choisiriez.